## 버스의 효율적인 운행를 위해 
# 퇴근시간 승차인원을 예측
## 하는 모델 개발

In [1]:
library('tidyverse')
library(lubridate)

ERROR: Error in library("tidyverse"): 'tidyverse'이라고 불리는 패키지가 없습니다


In [2]:
library(magrittr)
library(dplyr)
library(rvest)


다음의 패키지를 부착합니다: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [2]:
# 날짜가 다름 train test
# 결측치 없음 train test
# target 의 분포도가 한곳으로 치우쳐져 있음 -> scaling?
# ID 컬럼을 위치로? : station_code가 unique한 컬럼임.. 일단!

In [3]:
# data engineering 시 train, test 똑같이 변경해줘야함을 잊지말거라
train=read.csv('C:/Users/2106-017/Downloads/229255_bus_riders_at_rush_hour_data/train.csv',fileEncoding='utf-8')
otrain=read.csv('C:/Users/2106-017/Downloads/229255_bus_riders_at_rush_hour_data/train.csv',fileEncoding='utf-8')

test=read.csv('C:/Users/2106-017/Downloads/229255_bus_riders_at_rush_hour_data/test.csv',fileEncoding='utf-8')
otest=read.csv('C:/Users/2106-017/Downloads/229255_bus_riders_at_rush_hour_data/test.csv',fileEncoding='utf-8')

In [4]:
str(train)
cat('\t---------------------------------\n')
str(test)

'data.frame':	415423 obs. of  21 variables:
 $ id            : int  0 1 2 3 4 5 6 7 8 9 ...
 $ date          : Factor w/ 30 levels "2019-09-01","2019-09-02",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ bus_route_id  : int  4270000 4270000 4270000 4270000 4270000 4270000 4270000 4270000 4270000 4270000 ...
 $ in_out        : Factor w/ 2 levels "시내","시외": 2 2 2 1 1 1 2 2 2 1 ...
 $ station_code  : int  344 357 432 1579 1646 1648 1660 1666 1726 2008 ...
 $ station_name  : Factor w/ 1961 levels "(구)구판장","(구)중문동주민센터",..: 1482 1823 1407 1432 1576 1159 1443 1613 41 798 ...
 $ latitude      : num  33.5 33.5 33.5 33.5 33.3 ...
 $ longitude     : num  126 126 126 126 126 ...
 $ X6.7_ride     : num  0 1 1 0 0 0 0 1 0 0 ...
 $ X7.8_ride     : num  1 4 1 17 0 0 1 0 0 0 ...
 $ X8.9_ride     : num  2 4 0 6 0 0 0 0 0 0 ...
 $ X9.10_ride    : num  5 2 2 26 0 0 0 0 0 0 ...
 $ X10.11_ride   : num  2 5 0 14 0 1 2 0 0 0 ...
 $ X11.12_ride   : num  6 6 0 16 0 0 0 0 1 0 ...
 $ X6.7_takeoff  : num  0 0 0 0 0 0 0 0 0 0 ...


# 전처리
# Feature Engineering

### 필요없는 컬럼 제거

In [ ]:
train = subset(train, select=-c(id, station_name, station_code) )
test = subset(test, select=-c(id, station_name, station_code) )

### 날짜 주말 라벨링

In [1]:
# data 컬럼 -> 날짜 -> 주말 평일로 라벨링
#train
train$weekday = ifelse ( (weekdays(as.Date(train$date)) %in% c('일요일','토요일')), 0,1) # 0 :주말 , 1: 평일
train$weekend = ifelse ( (weekdays(as.Date(train$date)) %in% c('일요일','토요일')), 1,0) # 1 :주말 , 0 : 평일
train = subset(train,select=-date)

#test
test$weekday = ifelse ( (weekdays(as.Date(test$date)) %in% c('일요일','토요일')), 0,1) # 0 :주말 , 1: 평일
test$weekend = ifelse ( (weekdays(as.Date(test$date)) %in% c('일요일','토요일')), 1,0) # 1 :주말 , 0 : 평일
test = subset(test,select=-date)

ERROR: Error in as.Date(train$date): 객체 'train'를 찾을 수 없습니다


### 월화수목금토일 라벨링

In [ ]:
# train
otrain$date=as.Date(otrain$date) # 날짜형으로 변환
otrain$weekdays = wday(otrain$date) # 요일로 변환
train = cbind(train, otrain$weekdays)
train = rename(train, "weekdays"="otrain$weekdays")
head(train)

# test
otest$date=as.Date(otest$date) # 날짜형으로 변환
otest$weekdays = wday(otest$date) # 요일로 변환
test = cbind(test, otest$weekdays)
test = rename(test, "weekdays"="otest$weekdays")
head(test)

### 시내,시외 라벨링

In [6]:
# in_out 컬럼 -> inout으로 라벨링 
#train
train$out = ifelse(train$in_out == '시외',1,0)
train$inn = ifelse(train$in_out == '시내',1,0)
train = subset(train,select=-in_out)

#test
test$out = ifelse(test$in_out == '시외',1,0)
test$inn = ifelse(test$in_out == '시내',1,0)
test = subset(test,select=-in_out)

### 승하차,퇴근 시간대 통일하기(승하차 시간을 두시간씩으로 합치기)

In [8]:
# 승차  하차 시간대 통합하기
# train
train$ride6_8 = train$X6.7_ride + train$X7.8_ride
train$ride8_10 = train$X8.9_ride + train$X9.10_ride
train$ride10_12 = train$X10.11_ride + train$X11.12_ride

train$off6_8 = train$X6.7_takeoff + train$X7.8_takeoff
train$off8_10 = train$X8.9_takeoff+ train$X9.10_takeoff
train$off10_12 = train$X10.11_takeoff + train$X11.12_takeoff
train = subset(train, select=-c(X6.7_ride, X7.8_ride, X8.9_ride, X9.10_ride,
                                X10.11_ride, X11.12_ride, X6.7_takeoff,
                                X7.8_takeoff, X8.9_takeoff, X9.10_takeoff,
                                X10.11_takeoff, X11.12_takeoff))

#test
test$ride6_8 = test$X6.7_ride + test$X7.8_ride
test$ride8_10 = test$X8.9_ride + test$X9.10_ride
test$ride10_12 = test$X10.11_ride + test$X11.12_ride

test$off6_8 = test$X6.7_takeoff + test$X7.8_takeoff
test$off8_10 = test$X8.9_takeoff+ test$X9.10_takeoff
test$off10_12 = test$X10.11_takeoff + test$X11.12_takeoff
test = subset(test, select=-c(X6.7_ride, X7.8_ride, X8.9_ride, X9.10_ride,
                                X10.11_ride, X11.12_ride, X6.7_takeoff,
                                X7.8_takeoff, X8.9_takeoff, X9.10_takeoff,
                                X10.11_takeoff, X11.12_takeoff))

### 제주도를 사방으로 나누어 위도, 경도 거리 측정해 컬럼에 추가

In [13]:
jeju=c(126.52969, 33.51411) # 제주 측정소 근처
gosan=c(126.16283, 33.29382) #고산 측정소 근처
seongsan=c(126.8802, 33.38677) #성산 측정소 근처
po=c(126.5653, 33.24616) #서귀포 측정소 근처

In [14]:
library(geosphere)

In [15]:
wheretrain = subset(otrain,select=c(latitude, longitude))
wheretest = subset(otest,select=c(latitude, longitude))

In [ ]:
length(row.names(wheretrain))
length(row.names(wheretest))

In [17]:
위도, 경도 
for (i in 1:length(row.names(wheretrain)) ) {
    # train
    train$dist_jeju=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),jeju)
    train$dist_gosan=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),gosan)
    train$dist_seongsan=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),seongsan)
    train$dist_po=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),po)
}

In [18]:
for (i in 1:length(row.names(wheretest)) )
{    #test
    test$dist_jeju=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),jeju)
    test$dist_gosan=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),gosan)
#     test$dist_seongsan=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),seongsan)
    test$dist_po=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),po)
}

### 사람이 많을 곳으로 예상되는 장소와 정류장의 거리를 측정해 컬럼에 추가

In [27]:
coords_jejusicheong1 = c( 126.53035,33.49892) #제주시청(광양방면)의 위도 경도
coords_jejuairport = c(126.49345,33.50661) #제주국제공항(구제주방면)의 위도 경도
coords_hallahosp = c(126.486,33.48963) #한라병원의 위도 경도
coords_rotary = c( 126.49678,33.49143) # 제주도청신제주로터리의 위도 경도
coords_jejucenterhigh = c(126.5392,33.48902)# 제주중앙여자고등학교의 위도 경도
coords_jejumarket = c(126.52706,33.51315 ) #동문시장의 위도 경도
coords_jejusclass = c(126.48141,33.47626) #제주고등학교/중흥S클래스의 위도 경도
coords_centerroad = c(126.5239,33.51073) #중앙로(국민은행)의 위도 경도
coords_fiveway = c( 126.48092,33.48667) # 노형오거리의 위도 경도
coords_law = c(126.53476,33.49363) 

In [28]:
for (i in 1:length(row.names(wheretrain)) ) {
    # train
    train$dist_jejusicheong1=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_jejusicheong1)
    train$dist_jejuairport=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_jejuairport)
    train$dist_hallahosp=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_hallahosp)
    train$dist_rotary=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_rotary)
    train$dist_jejucenterhigh=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_jejucenterhigh)
    train$dist_jejumarket=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_jejumarket)
    train$dist_jejusclass=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_jejusclass)
    train$dist_centerroad=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_centerroad)
    train$dist_fiveway=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_fiveway)
    train$dist_law=distGeo(c(wheretrain$longitude[i], wheretrain$latitude[i]),coords_law)
}

In [29]:
for (i in 1:length(row.names(wheretest)) ) {
    # test
    test$dist_jejusicheong1=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_jejusicheong1)
    test$dist_jejuairport=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_jejuairport)
    test$dist_hallahosp=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_hallahosp)
    test$dist_rotary=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_rotary)
    test$dist_jejucenterhigh=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_jejucenterhigh)
    test$dist_jejumarket=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_jejumarket)
    test$dist_jejusclass=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_jejusclass)
    test$dist_centerroad=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_centerroad)
    test$dist_fiveway=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_fiveway)
    test$dist_law=distGeo(c(wheretest$longitude[i], wheretest$latitude[i]),coords_law)
}

### 요일별로 몇 명이 타고 내렸는지 컬럼 추가

In [ ]:
bts = read.csv('bus_bts.csv', fileEncoding='utf-8')
tail(bts)

In [ ]:
# 요일별로 몇 명 탔는지 : geton_date, user_count(누적ㅎ여함)
bts$geton_date = as.Date(bts$geton_date) # 날짜형으로 변환
bts$onweekdays = wday(bts$geton_date) # 요일형으로 변환
geton_day = aggregate(user_count~onweekdays, bts, sum)
geton_day = rename(geton_day, 'on_people'="user_count")

# 요일별로 몇 명 내렸는지 : getoff_date, user_count(누적ㅎ여함)
bts$getoff_date = as.Date(bts$getoff_date) # 날짜형으로 변환
bts$offweekdays = wday(bts$getoff_date) # 요일형으로 변환
getoff_day = aggregate(user_count~offweekdays, bts, sum)
getoff_day = rename(getoff_day, 'off_people'="user_count")

In [ ]:
train = merge(train, geton_day, by.x = 'weekdays', by.y='onweekdays')
train = merge(train, getoff_day, by.x = 'weekdays', by.y='offweekdays')

In [ ]:
test = merge(test, geton_day, by.x = 'weekdays', by.y='onweekdays')
test = merge(test, getoff_day, by.x = 'weekdays', by.y='offweekdays')

# 머신러닝 돌리면 시간이 너무 오래걸려서 따로 저장

In [33]:
write.csv(train, 'train1.csv',fileEncoding = 'utf-8' )
write.csv(test, 'test1.csv', fileEncoding = 'utf-8')